In [103]:
import time
import requests
import io
import os
import time 
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote_plus  

In [104]:
def get_google_images_url(query):
    # Encode the query to ensure it's URL-safe
    encoded_query = quote_plus(query)
    return f"https://www.google.com/search?hl=en&tbm=isch&q={encoded_query}"

In [105]:
def get_imgs_from_google(driver,max_num_imgs,url):
    driver.get(url)
    img_urls=set()
    l = driver.find_element("id",'W0wltc')
    l.click()
    # imgs = driver.find_elements(By.CLASS_NAME, "mNsIhb")
    while len(img_urls) < max_num_imgs:
        imgs = driver.find_elements(By.CLASS_NAME, "mNsIhb")
        for i,img in enumerate(imgs[len(img_urls):], start=len(img_urls)):
            actions = ActionChains(driver)
            actions.move_to_element(img).perform()
            time.sleep(2)  # Wait for overlay elements to load
            driver.execute_script("arguments[0].click();", img)
            try:
                images = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".sFlh5c.FyHeAf.iPVvYb"))
                    )
            except Exception as e:
                print(f"{i}Error clicking img: {e}")
                continue
            img_url = images.get_attribute("src")
            img_urls.add(img_url)
            print(f"{i}/{max_num_imgs}-->{img_url}")
            if len(img_urls) >= max_num_imgs:
                print(f"Collected {max_num_imgs} images. Exiting loop.")
                break  
    driver.quit()
    return img_urls




In [106]:
def download_image (download_path, urls,prefix):
    if not os.path.exists(download_path):
        os.makedirs(download_path)  # Creates the directory if it doesn't exist
    for i , url in enumerate(urls):
        try:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            file_path = os.path.join(download_path, f"{prefix}_{i + 1}.jpg")
                        # Convert image to RGB if it's in RGBA or P mode
            if image.mode in ("RGBA", "P"):
                image = image.convert("RGB")
            with open(file_path,"wb") as f :
                image.save(f, "JPEG")
        except Exception as e:
            print("FAILED -",e)

In [107]:
driver = webdriver.Chrome()  
search ="robots pictures"
url_robots = get_google_images_url(search)
urls_robots = get_imgs_from_google(driver,200,url_robots)
download_image("images//Robots",urls_robots,"robot")

0/200-->https://www.rolandberger.com/img/og/Roland_Berger-24_2195_Humanoid_robots-OG.jpg
1/200-->https://www.orientsoftware.com/Themes/Content/Images/blog/2022-01-07/robots-in-everyday-life.png
2/200-->https://media.licdn.com/dms/image/D4D12AQH1pZCGpcCr7w/article-cover_image-shrink_720_1280/0/1677021806463?e=2147483647&v=beta&t=XKkiYCvkNsBKpe0zf86t4WM0dxGQWZOGgO30UTj4LZU
3/200-->https://www.generationrobots.com/blog/wp-content/uploads/2024/02/banniere-blog-robots-humanoides-1.png
4/200-->https://media.istockphoto.com/id/1414699113/photo/small-robot-assistant-work-with-graphic-display.jpg?s=612x612&w=0&k=20&c=gGfba4h97L1tFjVWkPTiZUlfNHtkrf0fHhsmkY4S5Ng=
5/200-->https://images.axios.com/3UwkS6fiHGkufieBnEjHgvnCJ9I=/651x0:2811x2160/1920x1920/2023/08/30/1693404250342.jpg
6/200-->https://media.istockphoto.com/id/1354827837/photo/human-like-a-robot-in-a-pensive-posture.jpg?s=612x612&w=0&k=20&c=Jv5T-ImY4GR0pjGaypcqE6WZbBz6vO0oIsUJqR3GZS8=
7/200-->https://provenrobotics.ai/wp-content/uploads/2

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF7913438A5+3004357]
	(No symbol) [0x00007FF790FD9970]
	(No symbol) [0x00007FF790E8582A]
	(No symbol) [0x00007FF790E725DC]
	(No symbol) [0x00007FF790E724A0]
	(No symbol) [0x00007FF790E881A1]
	(No symbol) [0x00007FF790F1B379]
	(No symbol) [0x00007FF790EFB983]
	(No symbol) [0x00007FF790EC7628]
	(No symbol) [0x00007FF790EC8791]
	GetHandleVerifier [0x00007FF79136A00D+3161901]
	GetHandleVerifier [0x00007FF7913BE060+3506048]
	GetHandleVerifier [0x00007FF7913B400D+3465005]
	GetHandleVerifier [0x00007FF791130EEB+830987]
	(No symbol) [0x00007FF790FE467F]
	(No symbol) [0x00007FF790FE09D4]
	(No symbol) [0x00007FF790FE0B6D]
	(No symbol) [0x00007FF790FD0149]
	BaseThreadInitThunk [0x00007FFEC842257D+29]
	RtlUserThreadStart [0x00007FFECA52AF08+40]


In [ ]:
driver = webdriver.Chrome()  
search ="people photography"
url_humans = get_google_images_url(search)
urls_humans = get_imgs_from_google(driver,200,url_humans)
download_image("images//Humans",urls_humans,"human")

0/3-->https://andrewstuder.com/wp-content/uploads/2020/04/AF3I3830-scaled.jpg
1/3-->https://iso.500px.com/wp-content/uploads/2020/08/stock-photo-240662573.jpg
2/3-->https://images.unsplash.com/photo-1544124094-8aea0374da93?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8Mnx8cG9ydHJhaXQlMjBwaG90b2dyYXBoeXxlbnwwfHwwfHx8MA%3D%3D
Collected 3 images. Exiting loop.
